# 네이버뷰 크롤링 후 DB 적재

In [1]:
#모듈 import
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas  as pd 
import psycopg2
from datetime import date, timedelta
import re

In [2]:
# 자동 스크롤다운 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(5)

#select문 만들기
def select_command(table) :
    command = 'select * from ' + table
    return command

#insert문 만들기
def insert_command(table) :
    command = 'insert into ' + table + ' (title, date, url, hashtag) values (%s, %s, %s, %s);'
    return command

'뷔스티에 원피스', '롱패딩', '숏패딩', '롱코트', '숏코트', '잠옷', '홈웨어', '트위드자켓', '블레이저 자켓', '싱글 자켓', '크롭 자켓', '레더 자켓', '루즈핏', '테일러드자켓', '린넨자켓', '반팔자켓', '라이더 자켓', '오버핏 자켓', '청자켓', '니트가디건', '볼레로', '아가일', '크롭가디건', '반팔가디건', '앙고라', '바람막이', '아노락', '윈드브레이커', '바시티자켓', '바시티점퍼', '야구점퍼', '여름아우터', '간절기아우터', '겨울아우터', '니트 맨투맨', '기모 맨투맨', '크롭 맨투맨', '남자 맨투맨', '반집업', '반팔 맨투맨', '크롭후드', '기모후드', '후드티', '남자 후드', '꾸안꾸', '플리스', '후리스', '반팔티', '크롭 반팔', '블라우스', '와이셔츠', '셔츠', '체크셔츠', '타이다이', '옥스퍼드셔츠', '남방', '슬리브티', '니트', '남자니트', '조끼', '크롭티', '봄슬랙스', '여름슬랙스', '가을슬랙스', '겨울슬랙스', '간절기슬랙스', '데님팬츠', '연청', '진청', '중청', '흑청', '그레이진', '와이드팬츠', '밴딩바지', '스키니진', '린넨팬츠', '반바지', '하이웨스트', '로우웨스트', '부츠컷', '치노팬츠', '트레이닝복', '트레이닝바지', '조거팬츠', '카고바지', '일자팬츠', '냉장고바지', '멜빵바지', '오버롤팬츠', '찢청', '미니스커트', '미디스커트', '롱스커트', '플레어스커트', 'A라인 스커트', 'H라인 스커트', '청치마', '테니스스커트', '프릴스커트', '레더 스커트', '랩스커트', '데님스커트', '플리츠스커트', '미니원피스', '미디원피스', '롱원피스', '나시원피스', '코르셋', '셔츠원피스', '벨벳원피스', '꽃원피스', '골지원피스', '골지티', '쉬폰원피스', '쉬폰블라우스', '린넨원피스', '카라원피스', '랩원피스', '니트원피스', '후드원피스', '코듀로이', '반팔원피스', '점프슈트', '봄룩북', '봄옷쇼핑', '봄코디', '봄옷하울', '여름룩북', '여름옷쇼핑', '여름코디', '여름하울', '가을룩북', '가을옷쇼핑', '가을코디', '가을하울', '겨울룩북', '겨울옷쇼핑', '겨울코디', '겨울하울', '캠퍼스룩', '아메카지룩', '프레피룩', '하객룩', '놈코어룩', '시티보이룩', '미니멀룩', '애슬레저룩', '데이트룩', '출근룩', '데일리룩'

'침대', '매트리스', '토퍼', '소파', '테이블', '식탁', '책상', '거실장', 'tv장', '서랍', '수납장', '진열장', '책장', '선반', '행거', '옷장', '의자', '화장대', '콘솔','거울', '침구세트', '이불', '베개', '토퍼', '패드', '매트리스커버', '블라인드', '롤스크린', '커튼', '러그', '카페트', '매트', '쿠션', '방석', '소파커버', '조명', 'led평판등', '천장등', '장스탠드', '단스탠드', '데스크스탠드', '무드등', '장식조명', '벽조명', '센서등', '형광등', '집꾸미기', '방꾸미기', '인테리어', '티비다이', '템바보드', '오픈수납장', '혼수가구', '게이밍의자', '진열장', '장식장', '책꽂이', '와인셀러', '캐비닛', '식탁', '데스크', '콰트로 책상', '주방상판', '프래그마', '사이드책상', '베드테이블', '매쉬 의자', '사무용의자', '학생의자', '공부용의자', '중역의자', '스툴', '헤드형의자', '회전의자', '회의실의자', '팔걸이의자', '발매트', '욕실매트', '주방매트', '놀이매트', '층간소음매트', '극세사매트', '현관매트', '소프트매트', '소음매트', '뽀송매트', '미끄럼방지매트', '사계절매트', '메모리폼', '플라스틱서랍', 

keyword_list = ['책추천', '책리뷰', '책소개', '인생책', '필독서', '인문서적', '에세이', '여행 책', '라이트노벨 책', '자기계발 책']

In [25]:
keyword_list = ['이동형서랍', '와이드서랍', '시스템서랍', '전자렌지선반', '칸칸이서랍', '친환경서랍', '다용도서랍', '레일서랍', '투명서랍']

In [26]:
driver = webdriver.Chrome('C:\chromedriver.exe')

for query_txt in keyword_list :

    #크롬드라이버 실행
    driver.get('http://www.naver.com')
    time.sleep(2)

    #키워드를 검색한 네이버창 접속
    element = driver.find_element_by_id("query")
    element.send_keys(query_txt)
    element.submit()

    # 뷰 클릭
    driver.find_element_by_link_text("VIEW").click()
    time.sleep(2)

    #필터 클릭
    driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
    time.sleep(2)

    #기간 1년 설정
    driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()

    time.sleep(3)

    no2 = [ ]           # 게시글 번호 컬럼
    title2 = [ ]        # 게시물 제목 컬럼
    bdate2 = [ ]        # 작성 일자 컬럼
    url2 = [ ]          # url 컬럼
    hashtag2 = [ ]      # 해시태그 컬럼

    no = 1

    #더이상 스크롤이 안되면 종료
    i = 1
    time.sleep(1)

    #현재 스크롤 위치를 last_height로 받아옴
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        #스크롤 한 번 실행 후 스크롤 위치를 new_height로 받아옴
        scroll_down(driver)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")

        #더이상 스크롤이 안되면 종료
        if new_height == last_height:
            break

        last_height = new_height
        i += 1


    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    view_list = soup.find('ul','lst_total').find_all('li')
    print(query_txt, '포스트 개수 :', len(view_list))

    for i in view_list :
        
        no2.append(no)                            # 게시물 번호 리스트에 추가
        print(no, end=' ')

        all_title = i.find_all('a')
        title = all_title[5].get_text( )          # 게시물 제목
        title2.append(title)                      # 게시물 제목 리스트에 추가

        time.sleep(2)

        bdate = i.find('span','sub_time sub_txt').get_text( )  # 작성일자
        bdate.replace('.', '-')
        bdate2.append(bdate)                     # 작성일자 리스트에 추가

        time.sleep(2)

        url = str(all_title[0])
        url = url.split('"')[5]                 # 게시물 url
        url2.append(url)                        # url 리스트에 추가

        time.sleep(2)

        # 카페글은 hashtag가 없다면 댓글을 보여줌
        try :
            outline_test = i.find('div', 'total_tag_area').get_text()    
            
            # hashtag가 안달려있는 블로그 글도 있기때문에 오류 방지
            try:
                hashtag = " ".join(re.findall("#[가-힣]{1,}", outline_test))
                hashtag2.append(hashtag)
                    
            except Exception as e:
                hashtag = ""
                hashtag2.append(hashtag)

        except : 
            hashtag = ""
            hashtag2.append(hashtag)


        if no == 200 : #한번에 200개 까지만 크롤링 가능
            break
            
        no += 1

    print(query_txt, '끝')

    bdate3 = [] #전처리 후 작성일자

    # 작성일자를 날짜형식으로 통일화
    for day in bdate2 :

        if '시간 전' in day :       # n시간 전 업로드 글
            day = date.today()
        elif '분 전' in day :       # n분 전 업로드 글
            day = date.today()
        elif '일 전' in day :       # n일 전 업로드 글 
            day = date.today() - timedelta(int(day[0])) #8일전 글 부터는 날짜로 반환
        elif day == '어제' :        # 어제 업로드 글
            day = date.today() - timedelta(1)
        else :
            day = day.rstrip('.')
            day = day.replace('.', '-')
        
        bdate3.append(day)

    # df화
    naver_blog = pd.DataFrame()
    naver_blog['제목'] = title2
    naver_blog['작성일자'] = bdate3
    naver_blog['주소'] = url2
    naver_blog['해시태그'] = hashtag2

    #date-time 타입으로 변환
    naver_blog['작성일자'] = pd.to_datetime(naver_blog['작성일자'])

    print(naver_blog.tail())

    #postgresql 접속
    conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()

    table = 'view_interior'

    #table에 데이터 넣기
    for i in range(len(naver_blog)) :
        cur.execute(insert_command(table), (naver_blog.iloc[i][0], naver_blog.iloc[i][1], naver_blog.iloc[i][2], naver_blog.iloc[i][3]))
        conn.commit()

    #postgresql 연결종료
    conn.cursor().close()
    conn.close()

C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\chromedriver.exe')
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:23: DeprecationW

이동형서랍 포스트 개수 : 118
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 이동형서랍 끝
                                          제목       작성일자  \
113                   아디다스 이동형 항공커버(바퀴O) 팔아요 2022-04-25   
114  [판매] 슈퍼랙 이동식 중량랙 1.5톤 고중량 바퀴장착 이동형 철제선반 2022-06-13   
115  [판매] 슈퍼랙 이동식 중량랙 1.5톤 고중량 바퀴장착 이동형 철제선반 2022-06-13   
116  [판매] 슈퍼랙 이동식 중량랙 1.5톤 고중량 바퀴장착 이동형 철제선반 2022-06-11   
117  [판매] 슈퍼랙 이동식 중량랙 1.5톤 고중량 바퀴장착 이동형 철제선반 2022-06-10   

                                               주소  \
113      https://cafe.naver.com/golmarket/9721515   
114  https://cafe.naver.com/joonggonara/922903873   
115  https://cafe.naver.com/joonggonara/922973328   
116  https://cafe.naver.com/joonggonara/92

C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()


와이드서랍 포스트 개수 : 30
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 와이드서랍 끝
                                                   제목       작성일자  \
25   [원목가구 공장 베이직가구] 계절변화로 인한 옷정리가 필요할때 속이깊은 수납가구로 해결 2022-05-19   
26     퍼니하우스 헤라 확장형 와이드 서랍 수납 책상 화장대 일반화장대, 그레이화이트 내용 2022-06-03   
27  자기계발 목공예 서랍레일 와이드서랍장 협탁 직접 도면 재단 하남 미사 군자역 여성목... 2022-04-11   
28                           가구스타 : 화이트 수납 와이드서랍장 화장대 2021-08-05   
29               하라데코 와이드 서랍 멀티 책상 정리함 4단, 화이트, 1개 후기 2022-06-09   

                                                   주소  \
25            https://cafe.naver.com/moms1004/1510148   
26  https://blog.naver.com/zclu6cugsoowy/222758504711   
27  https://blog.naver.com/silverline0526/22269806...   
28              https://cafe.naver.com/rainup/2611533   
29  https://blog.naver.com/vgvy8lyfjmtac/222766403177   

                                                 해시태그  
25  #베이직가구 #베이직가구옷장 #베이직가구장롱 #베이직가구입본장 #베이직가구추천 #원...  
26                           

C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()


...

다용도서랍 포스트 개수 : 30
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 다용도서랍 끝
                                                   제목       작성일자  \
25                              30칸 다용도 수납장+ 서랍 3개 제작 2022-06-03   
26               [★생활용품★] 한샘 S.O.K 팬트리 핸디 다용도 수납박스 3P 2021-10-10   
27  키친 서랍 다용도 정리 트레이로 주방도구 완벽하게 정리했어요~ 주방도구 정리 완벽 ... 2021-09-14   
28                                이동식 트롤리에 슬라이드 서랍 달기 2022-01-24   
29      [신혼가구, 입주가구 전문 베이직가구] 활용도 높은 다용도 수납공간, 원목 렌지대 2021-12-01   

                                               주소  \
25    https://blog.naver.com/elihome/222758440383   
26            https://cafe.naver.com/tpmom/808651   
27  https://blog.naver.com/luvlriwon/222505367651   
28  https://blog.naver.com/kkomataku/222630089475   
29      https://cafe.naver.com/kyungmammo/5441699   

                                                 해시태그  
25                                                     
26                                                   

C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()


레일서랍 포스트 개수 : 30
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 레일서랍 끝
                                                   제목       작성일자  \
25  광주 싱크대, 오포 문형리 싱크대 상부장 재설치 [싱크대렌지후드, 배수구통, 수전,... 2022-03-24   
26             [성남시] 서랍 레일 고장이 나서 레일을 교체/싱크대 하부 서랍 수리 2022-05-25   
27  탑실마을 대주피오레2단지아파트 콘센트교체, 싱크대문짝수리, 서랍레일교체, 문짝경첩교... 2021-12-16   
28                         송파 방이동 옷장 서랍레일 출장 수리 다녀왔어요 2022-05-30   
29                             서랍레일 교체완료(류님~전동드릴 감사요) 2021-12-20   

                                                주소  \
25     https://blog.naver.com/jch5311/222666773481   
26      https://blog.naver.com/joick5/222744220124   
27  https://blog.naver.com/classwar11/222596720483   
28     https://blog.naver.com/pdb1914/222751859707   
29         https://cafe.naver.com/kdnstorys/550507   

                                                 해시태그  
25      #싱크대수리 #싱크대리폼 #상부장 #하부장 #배수구 #렌지후드 #문짝 #양념인출망  
26  #싱크대부분수리 #싱크대서랍레일교체 #고장이나고망가진서랍레일교체 #부분교체 #인덕

C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_13796/2412222373.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()


투명서랍 포스트 개수 : 30
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 투명서랍 끝
                                                  제목       작성일자  \
25                       주방 싱크대 서랍 정리 각양각색 일회용품 정리수납 2022-03-03   
26             [1년 전 오늘] 하루 하나 정리>투명컵을 이용한 부엌서랍 정리팁! 2021-09-28   
27                          책상서랍정리 트레이 이케아 책상에도 찰떡이네 2022-02-13   
28  hillo 다용도 싱크대 주방 서랍 방수 매트 4p, 투명 (21662368-7dda) 2022-04-10   
29                      한샘세이즈드레스룸으로 쉽게 드레스룸서랍정리했어요 ~ 2022-04-03   

                                                  주소  \
25  https://blog.naver.com/ilovezzangv2/222662689481   
26         https://blog.naver.com/logos/222519109444   
27     https://blog.naver.com/didvk1318/222646237431   
28       https://blog.naver.com/rzngy2w/222696506793   
29       https://blog.naver.com/000117k/222690503593   

                                                 해시태그  
25  #주방서랍정리 #서랍정리 #정리함 #수납함 #정리수납 #일회용품수납 #일회용품정리 ...  
26                                         

#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

In [ ]:
#table 확인하기
cur.execute(select_command(table))
conn.commit()
cur.fetchall()

In [17]:
#db 연결 종료
conn.cursor().close()
conn.close()

In [23]:
#varchar(500) type에 안들어가면 실행
b = 0

for i in range(len(url2)) :
    a = len(url2[i])

    if a > b :
        b = a
        print(i, b)

print(title2[20])

0 44
2 45
4 46
8 101
9 103
87 211
88 216
플라스틱 수저분리함/ 300 400 500 600 900... 칸칸 싱크대 서랍정리 수저함 트레이 수납장 소개입니다.


In [24]:
#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

table = 'view_interior'

#table에 데이터 넣기
for i in range(len(naver_blog)) :
    cur.execute(insert_command(table), (naver_blog.iloc[i][0], naver_blog.iloc[i][1], naver_blog.iloc[i][2], naver_blog.iloc[i][3]))
    conn.commit()

#postgresql 연결종료
conn.cursor().close()
conn.close()